In [ ]:
import os
import pandas as pd
from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom import minidom

# SSH and file details
REMOTE_SERVER = "10.7.8.64"
PASSWORD = "QAtr@de442"
REMOTE_LOG_PATH = "/home/scalp/load_balancing"
LOCAL_LOG_PATH = "/home/scalp/script/automation/load_balancing"
REMOTE_CFG_PATH = "/home/scalp/opra/opra.cfg"

# Fetch the log file from the remote server
os.system(f"sshpass -p {PASSWORD} scp {REMOTE_SERVER}:{REMOTE_LOG_PATH} {LOCAL_LOG_PATH}")

def extract_daily_packet_counts(logfile):
    # This function mimics the behavior of the bash script you provided earlier
    # It extracts the daily packet counts from the gmac log file
    # Returns a dataframe with the packet counts

    with open(logfile, 'r') as f:
        lines = f.readlines()

    data = {
        'line_number': [],
        'packets_received': []
    }

    # Extracting relevant lines
    for line in lines:
        if line.startswith(str(pd.Timestamp.now().date() - pd.Timedelta(days=1))):
            parts = line.strip().split()
            if "RCVD_PACKETS" in parts:
                line_number = int(parts[3].split('.')[3])
                packets_received = int(parts[-1])
                data['line_number'].append(line_number)
                data['packets_received'].append(packets_received)

    return pd.DataFrame(data)

def generate_opra_cfg(bins):
    # This function generates the opra.cfg file based on the bin-packing results
    # It returns the XML string

    root = Element('config')
    gmac = SubElement(root, 'gmac')
    SubElement(gmac, 'verbosity').text = "INFO"
    SubElement(gmac, 'logfile').text = "/home/scalp/opra/logs/gmac.log"
    SubElement(gmac, 'bufferpool').text = "10485760 128"
    SubElement(gmac, 'recovery').text = "off"
    SubElement(gmac, 'subscription').text = "on"

    markets = SubElement(gmac, 'markets')
    market = SubElement(markets, 'market', {'id': '0', 'type': 'OPRA_BINARY'})
    SubElement(market, 'verbosity').text = "INFO"
    SubElement(market, 'max-channel-buffer').text = "0"
    SubElement(market, 'max-hw-waiting-packets').text = "524288"
    SubElement(market, 'time-zone').text = "-4"
    SubElement(market, 'debug-mask').text = "INIT:STAT:BUF"

    channels = SubElement(market, 'channels')

    for core_num, bin_channels in enumerate(bins):
        for line_num in bin_channels:
            ip_address = f"224.0.204.{line_num}"
            port = f"450{line_num:02}"
            clx = f"clx{core_num % 4}"  # Wrap around after clx3
            channel_text = f"{line_num}  {core_num}  clxng {clx} {ip_address}:{port}:u"
            SubElement(channels, 'channel').text = channel_text

    return prettify(root)

# Main function that automates the whole process
def automate_process(logfile, output_file):
    # Extract daily packet counts
    df = extract_daily_packet_counts(logfile)

    # Run bin packing
    bins = bin_packing(df, num_cores=16)

    # Generate opra.cfg
    xml_str = generate_opra_cfg(bins)

    # Save to output file
    with open(output_file, 'w') as f:
        f.write(xml_str)
        
    # After generating the opra.cfg, send it back to the remote server (if needed)
    os.system(f"sshpass -p {PASSWORD} scp new_opra.cfg {REMOTE_SERVER}:{REMOTE_CFG_PATH}")

# Assume logs/msgstats is the input and new_opra.cfg is the output
automate_process('logs/msgstats', 'new_opra.cfg')